In [1]:
# import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import jensenshannon as JS
import matplotlib.transforms as mtransforms
from google.colab import files

## Get data for different climate types

In [2]:
## authenticate with GEE
import ee
ee.Authenticate()
ee.Initialize(project='15647375014')

In [3]:
# get gridmet data for site list

geos1 = {
          'Csa-Mediterranean':[-123.65, 43],
          'Cfb-Marine W Coast':[-122, 48.6],
          'Cfa-Humid Subtropical':[-122, 39],
          'Bsk-Semiarid Steppe':[-103.29, 39.18],
          'Dfb-Humid Continental (Cool Summer)':[-90.2, 45.81],
          'Dfa-Humid Continental (Warm Summer)':[-97.67, 41.5],
          'Cfa-Humid Subtropic':[-82.3, 32.8],
          'Bwh-Midlatitude Desert':[-112.85, 32.76],
          'Aw-Tropical':[-80.97, 25.36]  ,
          # 'Rivendell':[-123.650509, 39.726037]
          }

geos = {}
for key in geos1.keys():
  geo_obj = ee.Geometry.Point(geos1[key])
  geos[key] = geo_obj

start="2012-10-01"
stop="2019-10-01"

precip = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select('pr').filterDate(start,stop)
tmmn = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select('tmmn').filterDate(start,stop)
tmmx = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select('tmmx').filterDate(start,stop)
pet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select('eto').filterDate(start,stop)

In [4]:
bands = {precip:'pr (mm)',
         tmmn:'tmin (K)',
         tmmx:'tmax (K)',
         pet:'pet (mm)'
         }

data = {}

for key in geos1.keys():
  sitedata = []
  for band in bands.keys():

    psite = band.toBands().reproject(
          crs='EPSG:4326',scale=1000
          ).reduceRegion(ee.Reducer.mean(),
                                  geos[key].buffer(10000), 1000).getInfo()
    psite = {'date': [p.split('_')[0] for p in psite.keys()],
            bands[band]: psite.values()}
    psite = pd.DataFrame.from_dict(psite)
    psite['date'] = [s[:4]+'-'+s[4:6]+'-'+s[6:] for s in psite.date]
    psite['date'] = pd.to_datetime(psite['date'])
    psite = psite.set_index('date')
    if len(sitedata)==0:
      sitedata = psite
    else:
      sitedata = sitedata.merge(psite,left_index=True,right_index=True)

  data[key] = sitedata

aridities = {key:float(data[key]['pet (mm)'].sum()/data[key]['pr (mm)'].sum()) for key in data.keys()}
aridities

{'Csa-Mediterranean': 1.0459375175543977,
 'Cfb-Marine W Coast': 0.2666077579021537,
 'Cfa-Humid Subtropical': 3.3776435395648705,
 'Bsk-Semiarid Steppe': 3.0802918783959132,
 'Dfb-Humid Continental (Cool Summer)': 0.8187554729076263,
 'Dfa-Humid Continental (Warm Summer)': 1.463387955005378,
 'Cfa-Humid Subtropic': 1.189399669067293,
 'Bwh-Midlatitude Desert': 13.182560017206955,
 'Aw-Tropical': 1.0864414199526546}

In [5]:
asynch_index = {}
for key in data.keys():
  d = data[key]
  JSS = [float(JS(d['pr (mm)'].iloc[i*30:i*30+365],d['pet (mm)'][:365])) for i in range(11)]
  asynch_index[key] = float((JS(d['pr (mm)'].iloc[:365],d['pet (mm)'][:365])-min(JSS))**.5)
asynch_index

{'Csa-Mediterranean': 0.33738972703825193,
 'Cfb-Marine W Coast': 0.34907451555269314,
 'Cfa-Humid Subtropical': 0.2790741216294147,
 'Bsk-Semiarid Steppe': 0.16632147247691684,
 'Dfb-Humid Continental (Cool Summer)': 0.12622121924117072,
 'Dfa-Humid Continental (Warm Summer)': 0.08863121346129696,
 'Cfa-Humid Subtropic': 0.15677530891598637,
 'Bwh-Midlatitude Desert': 0.13718010527695287,
 'Aw-Tropical': 0.17211819555046334}

In [6]:
# data['Rivendell'].to_csv('data_Rivendell.csv')

In [ ]:
site_df = {'climate_type':list(geos1.keys()),
           'Longitude':[geos1[key][0] for key in geos1.keys()],
           'Latitude':[geos1[key][1] for key in geos1.keys()],
           'Aridity':[aridities[key] for key in geos1.keys()],
           'P [mm]':[float(data[key]['pr (mm)'].sum()/2) for key in geos1.keys()],
           'PET [mm]':[float(data[key]['pet (mm)'].sum()/2) for key in geos1.keys()],
           'Asynch':[asynch_index[key] for key in geos1.keys()]}
site_df = pd.DataFrame.from_dict(site_df)
site_df.to_csv('site_info.csv',index=False)

## Install HYDRUS

In [ ]:
# install HYDRUS
!pip install -q phydrus
# grab HYDRUS source code
!git clone https://github.com/phydrus/source_code
!git clone https://github.com/phydrus/phydrus



Cloning into 'source_code'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 6), reused 21 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 2.55 MiB | 6.43 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Cloning into 'phydrus'...
remote: Enumerating objects: 1004, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 1004 (delta 127), reused 133 (delta 119), pack-reused 855 (from 1)
Receiving objects: 100% (1004/1004), 8.49 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (574/574), done.


In [ ]:
%%bash
cd /content/source_code
cp ./source/* .
make
cp -r /content/phydrus/examples/phydrus_paper/* /content

/usr/bin/gfortran -g -ffpe-summary=none -c HYDRUS.FOR INPUT.FOR HYSTER.FOR MATERIAL.FOR OUTPUT.FOR SINK.FOR SOLUTE.FOR TEMPER.FOR TIME.FOR WATFLOW.FOR
/usr/bin/gfortran -o hydrus HYDRUS.o INPUT.o HYSTER.o MATERIAL.o OUTPUT.o SINK.o SOLUTE.o TEMPER.o TIME.o WATFLOW.o
rm HYDRUS.o INPUT.o HYSTER.o MATERIAL.o OUTPUT.o SINK.o SOLUTE.o TEMPER.o TIME.o WATFLOW.o


INPUT.FOR:271:72:

  271 |       if(j-n) 13,18,14
      |                                                                        1
INPUT.FOR:1603:72:

 1603 |         do 12 jS=1,NSD
      |                                                                        1
MATERIAL.FOR:814:72:

  814 |       if(X-2.0) 14,14,13
      |                                                                        1
MATERIAL.FOR:815:72:

  815 | 12    if(X-2.0) 16,16,13
      |                                                                        1
MATERIAL.FOR:819:72:

  819 | 14    if(X-1.0) 15,17,16
      |                                                                        1


In [ ]:
import phydrus as ps

## Set up and run HYDRUS for each climate type

In [ ]:
# set up and run HYDRUS for each site

for key in list(data.keys()):
  dd = data[key]

  exe = '/content/source_code/hydrus'
  # Folder for Hydrus files to be stored
  #Folder where the Hydrus files are to be stored
  ws = "/content/output"

  # Create model
  ml = ps.Model(exe_name=exe, ws_name=ws, name="model",
                mass_units="mmol", time_unit="days", length_unit="cm")

  atm = pd.read_csv("./data/atmosphere.csv", index_col=0)

  ml.add_time_info(tinit=0, tmax=len(atm)+1,print_times=True)
  ml.add_waterflow(top_bc=3, bot_bc=4)

  # Add the process solute transport
  ml.add_solute_transport(lupw=True)

  # Add materials - soil params from Jesse - best fit Rivendell?
  m = ml.get_empty_material_df(n=1)
  m.loc[0:1] = [[0.218, 0.520, 0.015, 2.03, 31.6, 0.5, 1.5, 30.0, 1, 0]]
  ml.add_material(m)

  # Define loop for potential root water uptake distribution proposed by Hoffman and Van Genuchten
  def z_loop(z, r1=10, r2=20):
      if z > -r1:
          return 1
      elif z < -(r1 + r2):
          return 0
      else:
          return(z + (r1 + r2)) / r2

  # bottom of each layer
  bottom = [-800]  # Depth of the soil column
  nodes = 800  # Dictretize into n nodes
  ihead = -100  # Determine initial pressure head
  ps.create_profile()

  # mat/lay options assign materials to layers?
  profile = ps.create_profile(bot=bottom, dx=abs(bottom[-1] / nodes), h=ihead, conc=1e-10)
  # root profile
  profile["Beta"] = profile.apply(lambda row: z_loop(row["x"],100,600), axis=1)
  ml.add_profile(profile)

  ml.add_root_uptake(model=0, crootmax=1e10, omegac=0, p2h=-1500, p2l=-1500, poptm=[-100])

  obs_depths = list(range(-10,-200,-10))
  ml.add_obs_nodes(obs_depths)

  sol = ml.get_empty_solute_df()
  sol.loc[:, "beta"] = 1.0
  ml.add_solute(sol,difw=1e-9)

  # set up atm file

  URL = 'https://docs.google.com/spreadsheets/d/1PQkx6K7Nxtwz5W94NwT9lldtzE4sTx4TVeKGguHuepQ/edit#gid=2126194800'
  sheet_url = URL.replace("/edit#gid=", "/export?format=csv&gid=")
  pet = pd.read_csv(sheet_url)
  pet['Timestamp'] = pd.to_datetime(pet['Timestamp'])
  pet = pet.set_index('Timestamp').rename(columns={'Air Temp WSAM [Angelo Meadow WS]':'rRoot'})
  pet = pet.loc['2016-01-01':'2018-01-01']
  pet = pet.resample('D').sum()
  pet['rRoot'] = dd['pet (mm)'].iloc[:len(pet)]/10
  pet['rSoil'] = 0
  pet['hCritA'] = 100000
  pet['tAtm'] = list(range(1,len(pet)+1))

  # change units to cm from mm
  dprecip = dd[['pet (mm)','pr (mm)']]/10
  # dprecip = dd.iloc[:len(atm)]
  # dprecip['idx'] = range(len(dprecip))
  # idx = dprecip['idx'].values
  # dates = dprecip.index.values

  # atm['Prec'] = dprecip.values
  # atm['rRoot'] = atm[['rRoot','rSoil']].sum(axis=1)
  # atm['rSoil'] = 0
  pet = pet.merge(dprecip,left_index=True,right_index=True,how='left').rename(columns={'pr (mm)':'Prec'})
  ml.add_atmospheric_bc(pet, hcrits=0)

  ml.profile['Conc'] = 0
  half = int(len(ml.profile)/10)
  ml.profile['Conc'].iloc[:half] = 1

  ml.write_atmosphere()

  dates = dd.index.values

  # Write the input and check if the model simulates
  ml.write_input()
  ml.simulate()

  # read HYDRUS outputs into dataframe

  d = ml.read_nod_inf(times=list(range(1,732)))
  for k in d.keys():
    d[k]['time'] = k
  alldata = [d[k] for k in d.keys()]
  alldata = pd.concat(alldata)

  alldata.to_csv('alldata_'+key+'.csv',index=False)
  files.download('alldata_'+key+'.csv')

  petdata = ml.read_tlevel()[['rRoot','rTop']]
  petdata['rTop'] = np.where(petdata.rTop<0,0,petdata.rTop)
  petdata = petdata.sum(axis=1).rename('PET').to_frame()
  petdata.to_csv('petdata_'+key+'.csv')
  files.download('petdata_'+key+'.csv')

  raindata = ml.atmosphere['Prec'].reset_index()
  raindata['time'] = list(range(1,len(raindata)+1))
  raindata.to_csv('raindata_'+key+'.csv')
  files.download('raindata_'+key+'.csv')

  etdata = ml.read_tlevel()[["vRoot"]]
  etdata = etdata.sum(axis=1).rename('vRoot')
  etdata.to_csv('etdata_'+key+'.csv')
  files.download('etdata_'+key+'.csv')

INFO: Directory /content/output created
INFO:phydrus.model:Directory /content/output created
/usr/local/lib/python3.11/dist-packages/phydrus/model.py:492: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5.96933338e-03 1.67171019e-01 2.45768050e-01 2.76599172e-01
 1.16111141e-01 9.84293164e-02 2.37543426e-01 4.15875560e-01
 1.34653221e-01 1.09186307e-01 4.90207275e-01 1.52105997e+00
 1.42040838e+00 1.37983025e+00 7.28743171e-01 1.79609426e+00
 2.23986872e+00 1.67309682e+00 1.49240530e+00 3.48588642e-01
 1.51879241e+00 7.79201329e-01 5.64448532e-01 3.69015177e-01
 5.80853991e-02 4.97918058e-02 9.01727337e-01 1.48506630e+00
 1.04074371e+00 3.66486778e-01 1.30788799e-01 4.28984197e-01
 3.39323320e-01 2.23400181e-01 1.56768553e-01 2.61580917e-01
 3.45904334e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.30536812e-02 4.27210119e-02 2.14438848e-01 4.36456812e-01
 1.25185053e-01 2.33440292e-03 2.06238842e-01 1.58106495

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.11/dist-packages/phydrus/read.py:214: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = read_csv(file, skiprows=s, nrows=e - s - 2, usecols=usecols,
/usr/local/lib/python3.11/dist-packages/phydrus/read.py:219: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.drop(index=data.index[0]).apply(to_numeric,


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.11/dist-packages/phydrus/read.py:214: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = read_csv(file, skiprows=s, nrows=e - s - 2, usecols=usecols,
/usr/local/lib/python3.11/dist-packages/phydrus/read.py:219: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.drop(index=data.index[0]).apply(to_numeric,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# files.download('alldata_Aw-Tropical.csv',)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

("Head",          "Moisture",       "K",                      "C",                 "Flux",      "Sink",         "Temp")

("Pressure Head", "Water Content", "Hydraulic Conductivity", "Hydraulic Capacity","Water Flux", "Root Uptake", "Temperature")

["h [{}]".format(l_unit), "Theta [-]", f"K [{l_unit}/days]",
                 f"C [1/{l_unit}]", f"v [{l_unit}/{t_unit}]",
                 f"S [1/{t_unit}]", "T [°C]"]